# Geração de labirintos com A* Manhattan

## Função A* Manhattan

In [ ]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import time
import tracemalloc
import heapq
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm

# Algoritmo de busca A*
def heuristica_manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_estrela_manhattan(labirinto, inicio, fim):
    altura, largura = len(labirinto), len(labirinto[0])
    aberta = []
    heapq.heappush(aberta, (0, inicio))
    veio_de = {}
    custo_ate_agora = {inicio: 0}
    nos_visitados = []

    tracemalloc.start()
    tempo_inicio = time.perf_counter()

    while aberta:
        _, atual = heapq.heappop(aberta)
        nos_visitados.append(atual)

        if atual == fim:
            break

        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = atual[0] + dx, atual[1] + dy
            vizinho = (nx, ny)
            if 0 <= nx < largura and 0 <= ny < altura and labirinto[ny][nx] == 1:
                novo_custo = custo_ate_agora[atual] + 1
                if vizinho not in custo_ate_agora or novo_custo < custo_ate_agora[vizinho]:
                    custo_ate_agora[vizinho] = novo_custo
                    prioridade = novo_custo + heuristica_manhattan(fim, vizinho)
                    heapq.heappush(aberta, (prioridade, vizinho))
                    veio_de[vizinho] = atual

    tempo_fim = time.perf_counter()
    memoria_usada = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    caminho = []
    atual = fim
    while atual != inicio:
        caminho.append(atual)
        atual = veio_de.get(atual)
        if atual is None:
            caminho = []
            break
    if caminho:
        caminho.append(inicio)
        caminho.reverse()

    return {
        "caminho": caminho,
        "nos_visitados": nos_visitados,
        "tempo_ms": (tempo_fim - tempo_inicio) * 1000,
        "memoria_bytes": memoria_usada,
        "comprimento": len(caminho),
        "qtd_nos_visitados": len(nos_visitados)
    }

## Geração de animações

In [1]:
def gerar_animacao(labirinto, resultado, caminho_saida, fps=10):
    # Aumentar a altura da figura para acomodar o título
    fig, ax = plt.subplots(figsize=(10, 10.2))  # Altura aumentada em 0.2

    cmap = plt.colormaps['viridis']
    norm = plt.Normalize(vmin=0, vmax=3)

    # Matriz de estado inicial
    estado = np.zeros_like(labirinto)
    estado[labirinto == 0] = 0  # Parede
    estado[labirinto == 1] = 1  # Livre

    img = ax.imshow(estado, cmap=cmap, norm=norm, interpolation='nearest')
    ax.axis('off')

    # Ajustes críticos para evitar corte
    plt.subplots_adjust(top=0.95)  # Reserva 5% do topo para o título
    fig.tight_layout(rect=[0, 0, 1, 0.95])  # Define área útil da figura

    writer = FFMpegWriter(fps=fps)
    total_nos = len(resultado['nos_visitados'])
    step = max(1, total_nos // 100)  # Máximo de 100 frames para exploração

    with writer.saving(fig, caminho_saida, dpi=100):
        # Frame inicial
        writer.grab_frame()

        # Fase de exploração
        for i in tqdm(range(0, total_nos, step), desc="Gerando frames"):
            bloco = resultado['nos_visitados'][i:i+step]
            for x, y in bloco:
                if estado[y][x] == 1:  # Só atualiza se for espaço livre
                    estado[y][x] = 3  # Visitado
            img.set_array(estado)
            ax.set_title(f"Exploração: {min(i+step, total_nos)}/{total_nos} nós", fontsize=10)  # Fonte reduzida
            writer.grab_frame()

        # Fase do caminho (se existir)
        if resultado['caminho']:
            for x, y in resultado['caminho']:
                estado[y][x] = 2  # Caminho
            img.set_array(estado)
            ax.set_title(f"Caminho encontrado! Comprimento: {len(resultado['caminho'])}", fontsize=10)
            for _ in range(30):  # 3 segundos de exibição final
                writer.grab_frame()
        else:
            ax.set_title("Caminho não encontrado!", fontsize=10)
            for _ in range(30):
                writer.grab_frame()

    plt.close(fig)

## Geração de imagens estáticas

In [ ]:
# Função para gerar imagens estáticas
def gerar_imagem(labirinto, resultado, caminho_saida, titulo):
    visualizacao = np.zeros_like(labirinto)
    visualizacao[labirinto == 0] = 0  # Parede
    visualizacao[labirinto == 1] = 1  # Livre

    # Marcar nós visitados (exceto caminho)
    for x, y in resultado['nos_visitados']:
        if visualizacao[y][x] == 1:
            visualizacao[y][x] = 3  # Visitado

    # Marcar caminho
    for x, y in resultado['caminho']:
        visualizacao[y][x] = 2  # Caminho

    plt.figure(figsize=(10, 10))
    plt.imshow(visualizacao, cmap=plt.colormaps['viridis'], vmin=0, vmax=3)
    plt.axis('off')
    plt.title(titulo)

    # Criar legenda
    legenda = [
        mpatches.Patch(color=plt.colormaps['viridis'](0/3), label='Parede'),
        mpatches.Patch(color=plt.colormaps['viridis'](1/3), label='Espaço livre'),
        mpatches.Patch(color=plt.colormaps['viridis'](2/3), label='Caminho final'),
        mpatches.Patch(color=plt.colormaps['viridis'](3/3), label='Nós visitados')
    ]
    plt.legend(handles=legenda, loc='upper right', bbox_to_anchor=(1.15, 1.0))

    plt.savefig(caminho_saida, bbox_inches='tight')
    plt.close()

## Configuração de pastas

In [ ]:
# Configurações principais
pastas = {
    "pequenos": "pequeno",
    "medios": "medio",
    "medio-grande": "medio-grande",
    "grandes": "grande"
}
abordagem = "astar_manhattan"

# Criar estrutura de diretórios
os.makedirs("midias", exist_ok=True)
for categoria in ["imagens", "animacoes"]:
    for tamanho in pastas.keys():
        os.makedirs(f"midias/{categoria}/{abordagem}/{tamanho}", exist_ok=True)


## Processamento dos labirintos

In [ ]:
# Processar todos os labirintos
for tamanho_pasta, tamanho_nome in pastas.items():
    print(f"\nProcessando labirintos {tamanho_nome}...")

    for i in range(1, 21):
        nome_arquivo = f"labirinto_{i:02d}.npy"
        lab_path = os.path.join("labirintos", tamanho_pasta, nome_arquivo)
        print(f"\nProcessando: {nome_arquivo}")

        try:
            # Carregar labirinto
            labirinto = np.load(lab_path)
            entrada = (0, 1)
            saida = (labirinto.shape[1]-1, labirinto.shape[0]-2)

            # Executar algoritmo
            resultado = a_estrela_manhattan(labirinto, entrada, saida)
            sucesso = bool(resultado['caminho'])

            # Gerar imagem estática
            img_path = f"midias/imagens/{abordagem}/{tamanho_pasta}/{nome_arquivo.replace('.npy', '.png')}"
            gerar_imagem(
                labirinto,
                resultado,
                img_path,
                f"Labirinto {tamanho_nome} - {nome_arquivo}"
            )
            print(f"Imagem gerada: {img_path}")

            # Gerar animação (apenas se encontrou caminho)
            if sucesso:
                anim_path = f"midias/animacoes/{abordagem}/{tamanho_pasta}/{nome_arquivo.replace('.npy', '.mp4')}"
                gerar_animacao(labirinto, resultado, anim_path)
                print(f"Animação gerada: {anim_path}")
            else:
                print("Caminho não encontrado. Animação não gerada.")

        except Exception as e:
            print(f"Erro no processamento: {str(e)}")

print("\nProcessamento concluído com sucesso!")


Processando labirintos pequeno...

Processando: labirinto_01.npy
Imagem gerada: midias/imagens/astar_manhattan/pequenos/labirinto_01.png


Gerando frames: 100%|██████████| 165/165 [00:27<00:00,  6.11it/s]


Animação gerada: midias/animacoes/astar_manhattan/pequenos/labirinto_01.mp4

Processando: labirinto_02.npy
Imagem gerada: midias/imagens/astar_manhattan/pequenos/labirinto_02.png


Gerando frames: 100%|██████████| 175/175 [00:23<00:00,  7.35it/s]


Animação gerada: midias/animacoes/astar_manhattan/pequenos/labirinto_02.mp4

Processando: labirinto_03.npy
Imagem gerada: midias/imagens/astar_manhattan/pequenos/labirinto_03.png


...

Animação gerada: midias/animacoes/astar_manhattan/grandes/labirinto_18.mp4

Processando: labirinto_19.npy
Imagem gerada: midias/imagens/astar_manhattan/grandes/labirinto_19.png


Gerando frames: 100%|██████████| 101/101 [00:23<00:00,  4.31it/s]


Animação gerada: midias/animacoes/astar_manhattan/grandes/labirinto_19.mp4

Processando: labirinto_20.npy
Imagem gerada: midias/imagens/astar_manhattan/grandes/labirinto_20.png


Gerando frames: 100%|██████████| 101/101 [00:25<00:00,  3.99it/s]


Animação gerada: midias/animacoes/astar_manhattan/grandes/labirinto_20.mp4

Processamento concluído com sucesso!


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("midias", "zip", "/content/midias")
files.download("midias.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>